In [1]:
import bqplot
import bqplot.pyplot as blt
import numpy as np
import ipywidgets as ipy
import pandas as pd

## Data prep

In [2]:
df = pd.DataFrame({
    "x": np.linspace(0, 10, 100),
    "y": np.sin(np.linspace(0, 10, 100))
})

In [3]:
df["selected"] = False

In [4]:
scales = {"x": bqplot.LinearScale(), "y": bqplot.LinearScale()}

## Current working prototype

In [5]:
scatter = bqplot.Scatter(
    x=df.loc[df["selected"] == False, "x"],
    y=df.loc[df["selected"] == False, "y"],
    scales=scales,
    colors=["blue"],
    opacity=[1]
)

selected_scatter = bqplot.Scatter(
    x=df.loc[df["selected"] == True, "x"],
    y=df.loc[df["selected"] == True, "y"],
    scales=scales,
    colors=["orange"],
    opacity=[1]
)

sel = bqplot.interacts.BrushSelector(
    x_scale=scales["x"], 
    y_scale=scales["y"],
    # marks required so that the mark itself can have
    # .selected attribute
    marks=[scatter, selected_scatter],
)

panzoom = bqplot.interacts.PanZoom(
    scales={"x": [scales["x"]]}
)

dropdown = ipy.Dropdown(
    description="Tool", options=["pan/zoom", "box select"],
    value="box select"
)

interactions = {
    "pan/zoom": panzoom,
    "box select": sel
}

x_ax = bqplot.Axis(label="X-axis", scale=scales["x"])
x_ay = bqplot.Axis(label="Y-axis", scale=scales["y"], orientation="vertical")

# Pass the Selector instance to the Figure
fig = bqplot.Figure(
    marks=[scatter, selected_scatter],
    axes=[x_ax, x_ay],
    title="Toggle select",
    interaction=interactions.get(dropdown.value),
)

toolbar = bqplot.Toolbar(figure=fig)

def update_toggle_selector(*args):
    if sel.brushing is True:
        pass
    else:
        # Update the manual "selected" toggle
        idxs_to_toggle = scatter.selected
        if idxs_to_toggle is not None:
            idxs_to_toggle_globalized = df[df["selected"] == False].index[idxs_to_toggle]
            df.loc[idxs_to_toggle_globalized, "selected"] = True
                
        # Update the manual "selected" toggle
        idxs_to_toggle = selected_scatter.selected
        if idxs_to_toggle is not None:
            idxs_to_toggle_globalized = df[df["selected"] == True].index[idxs_to_toggle]
            df.loc[idxs_to_toggle_globalized, "selected"] = False

        # Refresh xy's
        # Tips from https://bqplot.readthedocs.io/en/latest/usage/updating-plots/ for hold_sync()
        with scatter.hold_sync():
            scatter.x = df.loc[df["selected"] == False, "x"]
            scatter.y = df.loc[df["selected"] == False, "y"]
            
        with selected_scatter.hold_sync():
            selected_scatter.x = df.loc[df["selected"] == True, "x"]
            selected_scatter.y = df.loc[df["selected"] == True, "y"]

def update_tool(*args):
    fig.interaction = interactions.get(dropdown.value)
            
# Trying to figure out how brushing works
html_div = ipy.HTML()        
        
def currently_selected(*args):
    html_div.value = f"""
    Currently scatter.selected: {scatter.selected} <br>
    Currently selected_scatter.selected: {selected_scatter.selected} <br>
    df["selected"].value_counts(): {df["selected"].value_counts()}
    """
    
scatter.observe(currently_selected, "selected")
dropdown.observe(update_tool, "value")
sel.observe(update_toggle_selector, "brushing")

ipy.VBox([dropdown, fig, html_div])

Panning is quite smooth here. The lag seen in the last notebook is probably coz of all of the diagrams being linked and updating at the same time.

## Refining pan and zoom

Saw this: https://github.com/bqplot/bqplot/issues/316#issuecomment-318619629, want it.

In [6]:
import traitlets
from collections import OrderedDict

In [7]:
scatter = bqplot.Scatter(
    x=df.loc[df["selected"] == False, "x"],
    y=df.loc[df["selected"] == False, "y"],
    scales=scales,
    colors=["blue"],
    opacity=[1]
)

selected_scatter = bqplot.Scatter(
    x=df.loc[df["selected"] == True, "x"],
    y=df.loc[df["selected"] == True, "y"],
    scales=scales,
    colors=["orange"],
    opacity=[1]
)

sel = bqplot.interacts.BrushSelector(
    x_scale=scales["x"], 
    y_scale=scales["y"],
    # marks required so that the mark itself can have
    # .selected attribute
    marks=[scatter, selected_scatter],
)

pz_x = bqplot.interacts.PanZoom(scales={"x": [scales["x"]]})
pz_y = bqplot.interacts.PanZoom(scales={"y": [scales["y"]]})
pz_xy = bqplot.interacts.PanZoom(scales={
    "x": [scales["x"]], "y": [scales["y"]], 
})

interactions = {
    "pan/zoom": panzoom,
    "box select": sel
}

interacts = ipy.ToggleButtons(
    options=OrderedDict([
        ('Selector', sel),
        ('xy ', pz_xy), 
        ('x ', pz_x), 
        ('y ', pz_y),   
    ]),
    icons = ["hand-pointer-o", "arrows", "arrows-h", "arrows-v"],
    tooltips = ["Select", "zoom/pan in x & y", "zoom/pan in x only", "zoom/pan in y only"],
)
interacts.style.button_width = "100px"

x_ax = bqplot.Axis(label="X-axis", scale=scales["x"])
x_ay = bqplot.Axis(label="Y-axis", scale=scales["y"], orientation="vertical")

# Pass the Selector instance to the Figure
fig = bqplot.Figure(
    marks=[scatter, selected_scatter],
    axes=[x_ax, x_ay],
    title="Toggle select",
    interaction=interacts.value,
)

toolbar = bqplot.Toolbar(figure=fig)

def update_toggle_selector(*args):
    if sel.brushing is True:
        pass
    else:
        # Update the manual "selected" toggle
        idxs_to_toggle = scatter.selected
        if idxs_to_toggle is not None:
            idxs_to_toggle_globalized = df[df["selected"] == False].index[idxs_to_toggle]
            df.loc[idxs_to_toggle_globalized, "selected"] = True
                
        # Update the manual "selected" toggle
        idxs_to_toggle = selected_scatter.selected
        if idxs_to_toggle is not None:
            idxs_to_toggle_globalized = df[df["selected"] == True].index[idxs_to_toggle]
            df.loc[idxs_to_toggle_globalized, "selected"] = False

        # Refresh xy's
        # Tips from https://bqplot.readthedocs.io/en/latest/usage/updating-plots/ for hold_sync()
        with scatter.hold_sync():
            scatter.x = df.loc[df["selected"] == False, "x"]
            scatter.y = df.loc[df["selected"] == False, "y"]
            
        with selected_scatter.hold_sync():
            selected_scatter.x = df.loc[df["selected"] == True, "x"]
            selected_scatter.y = df.loc[df["selected"] == True, "y"]
            

reset_zoom_btn = ipy.Button(
    description='Reset zoom',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reset zoom',
    icon='arrows-alt'
)

def perform_zoom_reset(*args):
    # Reset the x and y axes on the figure
    fig.axes[0].scale.min = None
    fig.axes[1].scale.min = None
    fig.axes[0].scale.max = None
    fig.axes[1].scale.max = None  
    
reset_zoom_btn.on_click(perform_zoom_reset)
reset_zoom_btn.layout.width = "120px"

html_div = ipy.HTML()
        
def currently_selected(*args):
    html_div.value = f"""
    Currently scatter.selected: {scatter.selected} <br>
    Currently selected_scatter.selected: {selected_scatter.selected} <br>
    df["selected"].value_counts(): {df["selected"].value_counts()}
    """
    
scatter.observe(currently_selected, "selected")

# Instead of using a dropdown, use buttons
# dropdown.observe(update_tool, "value")
traitlets.link((interacts, "value"), (fig, "interaction"))

sel.observe(update_toggle_selector, "brushing")

ipy.VBox([
    fig, 
    interacts, 
    reset_zoom_btn,
    html_div],
)

Like it.

~Okay, gonna think more on the ipywidgets I can use to spiff things up a bit more. Probably done with the bqplot part? Left with the widgets part and wrapping things up.~ Nah

## Adding refinements to the UI

In [32]:
scatter = bqplot.Scatter(
    x=df.loc[df["selected"] == False, "x"],
    y=df.loc[df["selected"] == False, "y"],
    scales=scales,
    colors=["gray"],
    # For some reason, opacity doesn't work
    opacity=np.array([0.05] * len(df.loc[df["selected"] == False])),
    # Don't need legend formatting if colors are clear
    # display_legend=True,
    # name="Not selected",
    # labels=["Not selected"] * len(df.loc[df["selected"] == False])
)

selected_scatter = bqplot.Scatter(
    x=df.loc[df["selected"] == True, "x"],
    y=df.loc[df["selected"] == True, "y"],
    scales=scales,
    colors=["orange"],
    opacity=[1],
    # Don't need legend formatting if colors are clear
    # display_legend=True,
    # name="Selected",
    # labels=["Selected"] * len(df.loc[df["selected"] == True])
)

sel = bqplot.interacts.BrushSelector(
    x_scale=scales["x"], 
    y_scale=scales["y"],
    # marks required so that the mark itself can have
    # .selected attribute
    marks=[scatter, selected_scatter],
)

# Inspired by this fine piece of code
# https://github.com/bqplot/bqplot/issues/316#issuecomment-318619629
pz_x = bqplot.interacts.PanZoom(scales={"x": [scales["x"]]})
pz_y = bqplot.interacts.PanZoom(scales={"y": [scales["y"]]})
pz_xy = bqplot.interacts.PanZoom(scales={
    "x": [scales["x"]], "y": [scales["y"]], 
})

interactions = {
    "pan/zoom": panzoom,
    "box select": sel
}

interacts = ipy.ToggleButtons(
    options=OrderedDict([
        ('Selector', sel),
        ('xy ', pz_xy), 
        ('x ', pz_x), 
        ('y ', pz_y),   
    ]),
    icons = ["hand-pointer-o", "arrows", "arrows-h", "arrows-v"],
    tooltips = ["Select", "zoom/pan in x & y", "zoom/pan in x only", "zoom/pan in y only"],
)
interacts.style.button_width = "100px"

x_ax = bqplot.Axis(label="X-axis", scale=scales["x"])
x_ay = bqplot.Axis(label="Y-axis", scale=scales["y"], orientation="vertical")

# For some reason, Tooltips don't show up
scatter.tooltip = bqplot.Tooltip(
    fields=["x", "y"],
    labels=["x", "y"],
    formats = ['','.2f']
)

selected_scatter.tooltip = bqplot.Tooltip(
    fields=["x", "y"],
    labels=["x", "y"],
)

# Pass the Selector instance to the Figure
fig = bqplot.Figure(
    marks=[scatter, selected_scatter],
    axes=[x_ax, x_ay],
    title="Toggle select",
    interaction=interacts.value,
)

toolbar = bqplot.Toolbar(figure=fig)

def update_toggle_selector(*args):
    if sel.brushing is True:
        pass
    else:
        # Update the manual "selected" toggle
        idxs_to_toggle = scatter.selected
        if idxs_to_toggle is not None:
            idxs_to_toggle_globalized = df[df["selected"] == False].index[idxs_to_toggle]
            df.loc[idxs_to_toggle_globalized, "selected"] = True
                
        # Update the manual "selected" toggle
        idxs_to_toggle = selected_scatter.selected
        if idxs_to_toggle is not None:
            idxs_to_toggle_globalized = df[df["selected"] == True].index[idxs_to_toggle]
            df.loc[idxs_to_toggle_globalized, "selected"] = False

        # Refresh xy's
        # Tips from https://bqplot.readthedocs.io/en/latest/usage/updating-plots/ for hold_sync()
        with scatter.hold_sync():
            scatter.x = df.loc[df["selected"] == False, "x"]
            scatter.y = df.loc[df["selected"] == False, "y"]
            
        with selected_scatter.hold_sync():
            selected_scatter.x = df.loc[df["selected"] == True, "x"]
            selected_scatter.y = df.loc[df["selected"] == True, "y"]
            

reset_zoom_btn = ipy.Button(
    description='Reset zoom',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reset zoom',
    icon='arrows-alt'
)

def perform_zoom_reset(*args):
    # Reset the x and y axes on the figure
    fig.axes[0].scale.min = None
    fig.axes[1].scale.min = None
    fig.axes[0].scale.max = None
    fig.axes[1].scale.max = None  
    
reset_zoom_btn.on_click(perform_zoom_reset)
reset_zoom_btn.layout.width = "120px"

html_div = ipy.HTML()
        
def currently_selected(*args):
    html_div.value = f"""
    Currently scatter.selected: {scatter.selected} <br>
    Currently selected_scatter.selected: {selected_scatter.selected} <br>
    df["selected"].value_counts(): {df["selected"].value_counts()}
    """
    
scatter.observe(currently_selected, "selected")

# Instead of using a dropdown, use buttons
# dropdown.observe(update_tool, "value")
traitlets.link((interacts, "value"), (fig, "interaction"))

sel.observe(update_toggle_selector, "brushing")

ipy.VBox([
    fig, 
    interacts, 
    reset_zoom_btn,
    html_div],
)

## Inching closer to a working prototype

What would the API look like?

```python
label(df=df, x_col="x", y_col="y", label_col_name="selected")
```

+ Edits the df in-place straight up, keeping things simple.

What info do I want to show in the interface?

+ Scrolling if there's lots of data?
+ Stats of selected thus far
+ Meaningful title
+ Visual aid: plotting other columns in the df if needed?
+ Support date x-axis?

Get started with the minimal usable version bah. Build out the API, check the title and push.

Restarted the notebook at this point for a clean slate.

In [1]:
import numpy as np
import pandas as pd
import traitlets
import bqplot
import ipywidgets as ipy
from collections import OrderedDict

In [32]:
def label(df, x_col="x", y_col="y", label_col_name="selected", title="nblabeller"):
    if label_col_name is None:
        label_col_name = "selected"
    
    if x_col is None:
        x_col = "x"
        
    if y_col is None:
        y_col = "y"
    
    if label_col_name in df.columns:
        # Whole col dtype is not bool, but np.dtype("bool")
        assert df[label_col_name].dtype is np.dtype("bool"), \
        f"Column {label_col_name} dtype should be bool, but is {df[label_col_name].dtype}. " + \
        "Use another label_col_name instead"
    else:
        df.loc[:, label_col_name] = False
    
    scales = {"x": bqplot.LinearScale(), "y": bqplot.LinearScale()}
    
    scatter = bqplot.Scatter(
        x=df.loc[df[label_col_name] == False, x_col],
        y=df.loc[df[label_col_name] == False, y_col],
        scales=scales,
        colors=["gray"],
        # For some reason, opacity doesn't work
        opacity=np.array([0.05] * len(df.loc[df[label_col_name] == False])),
        # Don't need legend formatting if colors are clear
        # display_legend=True,
        # name="Not selected",
        # labels=["Not selected"] * len(df.loc[df["selected"] == False])
    )

    selected_scatter = bqplot.Scatter(
        x=df.loc[df[label_col_name] == True, x_col],
        y=df.loc[df[label_col_name] == True, y_col],
        scales=scales,
        colors=["orange"],
        opacity=[1],
        # Don't need legend formatting if colors are clear
        # display_legend=True,
        # name="Selected",
        # labels=["Selected"] * len(df.loc[df["selected"] == True])
    )

    sel = bqplot.interacts.BrushSelector(
        x_scale=scales["x"], 
        y_scale=scales["y"],
        # marks required so that the mark itself can have
        # .selected attribute
        marks=[scatter, selected_scatter],
    )

    # Inspired by this fine piece of code
    # https://github.com/bqplot/bqplot/issues/316#issuecomment-318619629
    pz_x = bqplot.interacts.PanZoom(scales={"x": [scales["x"]]})
    pz_y = bqplot.interacts.PanZoom(scales={"y": [scales["y"]]})
    pz_xy = bqplot.interacts.PanZoom(scales={
        "x": [scales["x"]], "y": [scales["y"]], 
    })

    interacts = ipy.ToggleButtons(
        options=OrderedDict([
            ('Selector', sel),
            ('xy ', pz_xy), 
            ('x ', pz_x), 
            ('y ', pz_y),   
        ]),
        icons = ["hand-pointer-o", "arrows", "arrows-h", "arrows-v"],
        tooltips = ["Select", "Zoom/pan in x & y", "Zoom/pan in x only", "Zoom/pan in y only"],
    )
    interacts.style.button_width = "100px"

    x_ax = bqplot.Axis(label=x_col, scale=scales["x"])
    x_ay = bqplot.Axis(label=y_col, scale=scales["y"], orientation="vertical")

    # For some reason, Tooltips don't show up
    scatter.tooltip = bqplot.Tooltip(
        fields=["x", "y"],
        labels=["x", "y"],
        formats = ['','.2f']
    )

    selected_scatter.tooltip = bqplot.Tooltip(
        fields=["x", "y"],
        labels=["x", "y"],
    )

    # Pass the Selector instance to the Figure
    fig = bqplot.Figure(
        marks=[scatter, selected_scatter],
        axes=[x_ax, x_ay],
        title=title,
        interaction=interacts.value,
    )

    def update_toggle_selector(*args):
        if sel.brushing is True:
            pass
        else:
            # Update the manual "selected" toggle
            idxs_to_toggle = scatter.selected
            if idxs_to_toggle is not None:
                idxs_to_toggle_globalized = df[df[label_col_name] == False].index[idxs_to_toggle]
                df.loc[idxs_to_toggle_globalized, label_col_name] = True

            # Update the manual "selected" toggle
            idxs_to_toggle = selected_scatter.selected
            if idxs_to_toggle is not None:
                idxs_to_toggle_globalized = df[df[label_col_name] == True].index[idxs_to_toggle]
                df.loc[idxs_to_toggle_globalized, label_col_name] = False

            # Refresh xy's
            # Tips from https://bqplot.readthedocs.io/en/latest/usage/updating-plots/ for hold_sync()
            with scatter.hold_sync():
                scatter.x = df.loc[df[label_col_name] == False, x_col]
                scatter.y = df.loc[df[label_col_name] == False, y_col]

            with selected_scatter.hold_sync():
                selected_scatter.x = df.loc[df[label_col_name] == True, x_col]
                selected_scatter.y = df.loc[df[label_col_name] == True, y_col]


    reset_zoom_btn = ipy.Button(
        description='Reset zoom',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Reset zoom',
        icon='arrows-alt'
    )

    def perform_zoom_reset(*args):
        # Reset the x and y axes on the figure
        fig.axes[0].scale.min = None
        fig.axes[1].scale.min = None
        fig.axes[0].scale.max = None
        fig.axes[1].scale.max = None  

    reset_zoom_btn.on_click(perform_zoom_reset)
    reset_zoom_btn.layout.width = "120px"

    def get_counts():
        val_counts = df[label_col_name].value_counts()
        if True in val_counts.index:
            true_counts = val_counts.loc[True]
        else:
            true_counts = 0
            
        if False in val_counts.index:
            false_counts = val_counts.loc[False]
        else:
            false_counts = 0
            
        return f"""
        Num pts selected: {true_counts} <br>
        Num pts unselected: {false_counts}
        """
    
    html_div = ipy.HTML(get_counts())

    def currently_selected(*args):
        html_div.value = get_counts()
        
    sel.observe(currently_selected, "selected")

    # Instead of using a dropdown, use buttons
    # dropdown.observe(update_tool, "value")
    traitlets.link((interacts, "value"), (fig, "interaction"))

    sel.observe(update_toggle_selector, "brushing")

    return ipy.VBox([
        fig, 
        interacts, 
        reset_zoom_btn,
        html_div],
    )

In [33]:
df = pd.DataFrame({
    "x": np.linspace(0, 10, 100),
    "y": np.sin(np.linspace(0, 10, 100))
})

In [34]:
label(df, x_col="x", y_col="y", label_col_name="selected", title="nblabeller")

Alright. Package this and test.